In [11]:
import json
import os
from collections import defaultdict
import numpy as np
import pandas as pd

from tira.third_party_integrations import ensure_pyterrier_is_loaded, persist_and_normalize_run
from tira.rest_api_client import Client
import pyterrier as pt

In [12]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split

In [13]:
ensure_pyterrier_is_loaded()
tira = Client()
pt_dataset = pt.get_dataset('irds:ir-lab-sose-2024/ir-acl-anthology-20240504-training')
topics = pt_dataset.get_topics('text')

In [14]:
qrels_df = pt_dataset.get_qrels()

# qrels_dict: {qid: {docno: label}}
qrels_dict = defaultdict(dict)

for i, row in qrels_df.iterrows():
    qrels_dict[row['qid']][row['docno']] = row['label']

print(len(qrels_df))

2623


In [15]:
if os.getcwd().endswith('pagerank'):
    os.chdir('..')

In [16]:
with open('data/docs_with_all_info.json', 'r') as file:
    documents = json.load(file)

In [17]:
df_docs = pd.DataFrame(documents)
df_docs = df_docs.transpose()

df_docs['docno'] = df_docs.index

df_docs['text'] = df_docs['title'] + ' ' + df_docs['abstract']
df_docs.drop(columns=['pagerank', 'title', 'abstract'], inplace=True)

df_docs.reset_index(drop=True, inplace=True)


df_docs = df_docs.to_dict(orient='records')


In [35]:
# My own index
import os

if not os.getcwd().endswith('pagerank'):
    os.chdir('pagerank')

print(os.getcwd())

#Create the index using PyTerrier
indexer = pt.IterDictIndexer(
    "../index/index_retrieved_docs",
    overwrite=True,
    fields=["text"],
    meta=["docno"]
)

# Index the documents
index = indexer.index(df_docs)

#Retrieve documents using BM25
bm25 = pt.BatchRetrieve(index, wmodel="BM25")

/workspaces/ir-lab-sose-2024-augsburg-information-retrieval/pagerank


In [33]:
if os.getcwd().endswith('pagerank'):
    os.chdir('..')

print(os.getcwd())

/workspaces/ir-lab-sose-2024-augsburg-information-retrieval


In [ ]:
# index = tira.pt.index('ir-lab-sose-2024/tira-ir-starter/Index (tira-ir-starter-pyterrier)', pt_dataset)

bm25 = pt.BatchRetrieve(index, wmodel="BM25")

bm25_run = bm25(pt_dataset.get_topics('text'))

In [37]:
print(len(bm25_run))

66391


In [38]:
bm25_dict = defaultdict(dict)

for i, row in bm25_run.iterrows():
    bm25_dict[row['qid']][row['docno']] = {'bm25_score': row['score'], 
                                           'bm25_rank': row['rank'], 
                                           'pagerank_score': documents[row['docno']]['pagerank']}

In [39]:
bm25_imputation = np.mean([scores['bm25_rank'] for doc_dict in bm25_dict.values() for scores in doc_dict.values()])
pagerank_imputation = np.mean([documents[docno]['pagerank'] for docno in documents])

print("BM25 Imputation: ", bm25_imputation)
print("Pagerank Imputation: ", pagerank_imputation)

BM25 Imputation:  497.66620475666883
Pagerank Imputation:  2.0335464700349214e-06


In [40]:
data_dict = defaultdict(dict)

for qid, doc_dict in qrels_dict.items():
    for docno, label in doc_dict.items():
        data_dict[qid][docno] = {}
        try:
            data_dict[qid][docno]['bm25_score'] = bm25_dict[qid][docno]['bm25_score']
        except:
            data_dict[qid][docno]['bm25_score'] =  bm25_imputation

        try:
            data_dict[qid][docno]['pagerank_score'] = bm25_dict[qid][docno]['pagerank_score']
        except:
            data_dict[qid][docno]['pagerank_score'] = pagerank_imputation
        
        try:
            data_dict[qid][docno]['label'] = qrels_dict[qid][docno]
        except:
            data_dict[qid][docno]['label'] = 0

In [ ]:
#Flatten the data
flattened_data = []
for qid, docs in data_dict.items():
    for docno, values in docs.items():
        flattened_data.append({
            'qid': qid,
            'docno': docno,
            'bm25': values['bm25_score'],
            'pagerank': values['pagerank_score'],
            'label': values['label']
        })

df = pd.DataFrame(flattened_data)

In [42]:
#Prepare training data
X = df[['bm25', 'pagerank']]
y = df['label']

In [43]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=420)

model = LogisticRegression()
model.fit(X, y)

LogisticRegression()

In [44]:
# Predict relevance scores
df['predicted_relevance'] = model.predict_proba(X)[:, 1]

In [45]:
# Select top 10 documents for each query
top_10_docs = df.sort_values(by=['qid', 'predicted_relevance'], ascending=[True, False]).groupby('qid').head(10)

In [46]:
# Save the resulting document orders in a JSON file
result = {}
for qid, group in top_10_docs.groupby('qid'):
    result[qid] = group[['docno', 'predicted_relevance']].to_dict(orient='records')

with open('top_10_docs.json', 'w') as f:
    json.dump(result, f, indent=4)

# Print learned weights
print("Learned weights for BM25 and PageRank:", model.coef_)

Top 10 documents for query 1:
                                           docno       bm25  pagerank  \
1             2019.tois_journal-ir0volumeA37A1.2   8.840354  0.000002   
3              2015.ipm_journal-ir0volumeA51A5.7   9.098527  0.000002   
39               1998.sigirconf_conference-98.22   9.323594  0.000003   
17                  2008.cikm_conference-2008.59   9.576598  0.000006   
40                 2009.cikm_conference-2009.277   9.730766  0.000002   
7                   2018.wsdm_conference-2018.47   9.777946  0.000002   
35  2018.sigirjournals_journal-ir0volumeA52A2.23   9.798317  0.000002   
12                  2008.cikm_conference-2008.76   9.991069  0.000002   
38             2003.sigirconf_conference-2003.49  10.078842  0.000004   
36   2012.sigirjournals_journal-ir0volumeA46A1.9  10.114001  0.000002   

    predicted_relevance  
1              0.575487  
3              0.575381  
39             0.575288  
17             0.575184  
40             0.575121  
7         

In [48]:
run = df[['qid', 'docno', 'predicted_relevance', 'bm25']]

run.rename(columns={'predicted_relevance': 'score'}, inplace=True)

run.sort_values(by=['qid', 'score'], ascending=[True, False], inplace=True)

run['rank'] = run.groupby('qid').cumcount()

print(run.head(15))

   qid                                         docno     score       bm25  \
1    1            2019.tois_journal-ir0volumeA37A1.2  0.575487   8.840354   
3    1             2015.ipm_journal-ir0volumeA51A5.7  0.575381   9.098527   
39   1               1998.sigirconf_conference-98.22  0.575288   9.323594   
17   1                  2008.cikm_conference-2008.59  0.575184   9.576598   
40   1                 2009.cikm_conference-2009.277  0.575121   9.730766   
7    1                  2018.wsdm_conference-2018.47  0.575101   9.777946   
35   1  2018.sigirjournals_journal-ir0volumeA52A2.23  0.575093   9.798317   
12   1                  2008.cikm_conference-2008.76  0.575013   9.991069   
38   1             2003.sigirconf_conference-2003.49  0.574977  10.078842   
36   1   2012.sigirjournals_journal-ir0volumeA46A1.9  0.574963  10.114001   
21   1  2010.sigirjournals_journal-ir0volumeA44A2.20  0.574962  10.116735   
16   1            2005.ipm_journal-ir0volumeA41A4.12  0.574850  10.386850   

/tmp/ipykernel_19574/3731424563.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  run.rename(columns={'predicted_relevance': 'score'}, inplace=True)
/tmp/ipykernel_19574/3731424563.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  run.sort_values(by=['qid', 'score'], ascending=[True, False], inplace=True)
/tmp/ipykernel_19574/3731424563.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-ver

In [49]:
eval = pt.Evaluate(run, qrels_df, metrics=["map", "ndcg", "ndcg_cut.10", "recip_rank", "recall_100"])
print("Evaluation Metrics:")
print(eval)

Evaluation Metrics:
{'map': 0.547369892273752, 'ndcg': 0.7323267767648299, 'ndcg_cut.10': 0.45129355824419753, 'recip_rank': 0.5550540146128383, 'recall_100': 0.9852941176470589}
